# PHYS20762 - Project 3 : Monte Carlo Techniques
# Penetration of Neutrons Through Shielding

Divij Gupta (ID: 10628702) <br>
University of Manchester <br>
April 2022

First we need to initialise the Python Interpreter

In [28]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.constants as const
import matplotlib.lines as mlines
import matplotlib.patches as mpatches

## Introduction

In this report, we look to simulate the motion and interaction of neutrons with materials of varying thicknesses to primarily see how many penetrate the 'shielding layer' and are transmitted but also to find how many are absorbed or reflected/black-scattered.  This is something that is important to consider in for example nuclear rector design to maintain the safety of personal working there. It is worth mentioning that we are only considering thermal neutrons here as these are what are used in nuclear reactors and because of certain assumptions we can make about them (detailed later).

The specifics of how the neutron motion is calculated is detailed as and when used below but the key component is the use of Monte Carlo integration techniques. We simulate a random walk for the neutrons in an attempt to recreate the uncertain nature of neutron interactions with a material.

## Background theory

We will first look at some background theory for a beam of neutrons travelling through a material. To start with we will only consider neutron absorption but will return to scattering which leads to reflection later.

The number of absorbing molecules per unit volume in any given material is given by:
$$ n = \frac{\rho N_{A}}{M_{r}} $$
where $\rho$ is the density of the material, $N_{A}$ is Avogadro's constant, and $M_{r}$ is the molar mass of the molecules in the material. From this we can find the number of neutrons that are absorbed in a thin layer of material as:
$$ N_{L} = n\sigma_{a} IL $$
where $L$ is the thickness of the material, $I$ is the intensity of incident neutrons, and $\sigma_{a}$ is the absorbing cross-section of the material.

Using this we can define the differential equation for the rate of absorption per unit thickness as:
$$ R = - \frac{dI}{dx} = n \sigma_{a} I $$
which is a first order ODE. Solving for the intensity we get:
$$ {I}(x) = I_{0} e^{-n\sigma x} $$
where $I_{0}$ is the initial intensity of radiation. Since intensity in this context is a measure of the number of incident neutrons, this equation shows how the number of neutrons varies as you go deeper into a material.

So far we have been working in microscopic cross-sections, $\sigma_{a}$, but if we instead define a macroscopic cross-section as:
$$ \Sigma_{a} = n\sigma_{a} $$
we can use it to define the mean free path using the equation:
$$ \lambda_{a} = \frac{1}{\Sigma_{a}} $$
which denotes the average distance a neutron can travel before being absorbed. Now using these equations we can redefine our intensity as:
$$ {I}(x) = I_{0} e^{-\frac{x}{\lambda_{a}}} $$
which is now given in terms of $\lambda_{a}$ thus making it much easier to work with.


A similar procedure can be applied to scattering where instead of subscript $a$ we now have subscript $s$ to denote scattering instead of absorption but form of the equations still hold as normal. For now, we just appreciate that this exists and focus on absorption but later on we will consider both processes in tandem.

The materials chosen here are water, lead and graphite although the procedure is valid for any material provided the required parameters for calculation are known.

In [29]:
# Storing all the given data plus some extra in a dictionary of dictionaries for each material for easy access later on.
neutron_data = {
    'water': {
        'absorption micro': 0.6652,   # Units of barns (10^-28 m^2 in SI units)
        'scattering micro': 103.0,    # Units of barns (10^-28 m^2 in SI units)
        'density': 1.00,              # Units of g/cm^3 (1000 kg/m^3 in SI units)
        'molar mass': 18.02,          # Units of g/mol (10^-3 kg/mol in SI units)
    },

    'lead': {
        'absorption micro': 0.158,
        'scattering micro': 11.221,
        'density': 11.35,
        'molar mass': 207.20,
    },

    'graphite': {
        'absorption micro': 0.0045,
        'scattering micro': 4.74,
        'density': 1.67,
        'molar mass': 12.01,
    },
}

# Creating a list of all materials so when looping through each material later, this list can be used instead of neutron_data
# .keys() every time - this is both simpler and computationally better
material_list = []
for test_material in neutron_data.keys():
    material_list.append(test_material)

# Setting up some potting information
get_ipython().magic('matplotlib notebook')
plt.rcParams.update({'font.size': 12})
plt.style.use('default')
plt.rcParams["figure.figsize"] = (7, 5)

C:\Users\divij\AppData\Local\Temp\ipykernel_264\2151726960.py:32: DeprecationWarning: `magic(...)` is deprecated since IPython 0.13 (warning added in 8.1), use run_line_magic(magic_name, parameter_s).
  get_ipython().magic('matplotlib notebook')


In [30]:
# Defining a function to find the parameters detailed in the theory above for a given material and to store them into the
# dictionary. This makes them both easy to access later on and saves having to calculate the parameters each time a calculation
# is performed
def find_material_parameters(material):

    # Defining the 2 events that can occur in the medium
    event_types = ['absorption', 'scattering']

    # Finding 'n' adding it to the dictionary
    density = neutron_data[material]['density']
    molar_mass = neutron_data[material]['molar mass']
    try:
        absorbing_molecule_number = density * const.N_A / molar_mass
    except ZeroDivisionError:
        if density == 0:
            absorbing_molecule_number = 0
        else:
            absorbing_molecule_number = np.inf
    neutron_data[material]['molecule number'] = absorbing_molecule_number

    for event_type in event_types:

        # Finding 'SIGMA' and adding it to the dictionary
        microscopic_cross_section = neutron_data[material][f'{event_type} micro']
        macroscopic_coss_section = microscopic_cross_section * absorbing_molecule_number
        neutron_data[material][f'{event_type} macro'] = macroscopic_coss_section

        # Finding 'lambda' and adding it to the dictionary
        try:
            lambda_test = 1 / macroscopic_coss_section
        except ZeroDivisionError:
            lambda_test = np.inf
        neutron_data[material][f'lambda {event_type}'] = lambda_test

# Finding the parameters for each of the 3 materials being studied
for test_material in material_list:
    find_material_parameters(test_material)

## Generating uniform random numbers in 1D

As mentioned before, using random numbers to generate random walks for neutrons is at the heart of this report. To that end we first need to investigate the generation of these random numbers and evaluate the effectiveness of the generation method.

What we need is a random uniform distribution to act as a basis for further non-uniform random numbers and to do this we will use the inbuilt numpy function, np.random.uniform(). In theory, this function produces a uniform set if random numbers over a given range but it is important to test how accurate this is and so we will perform this random sampling a number of times. Since this is simply a test, all we need is a 1D sample range with a reasonable number of neutrons.

To see the fluctuations between successive trials, we use histograms to first count and plot how many numbers fall within each bin for each trial and then average over all the trials to plot the mean count and observed error which should be binomial in nature as we are taking the standard deviation. To compare the effectiveness of the method, we also plot the expected mean and the expected binomial distribution error:
$$ \sigma^2 = np(1-p) $$
where $p$ is in this case given by $\frac{1}{bins}$. For a large enough sample size the error on each bin can be approximated as a normal distribution but we have chosen to stick with exact values for a better comparison.

It is important to note that while these numbers may seem random, they are not truly random and there is actually no such thing as a truly random number generator. The reason this doesn't matter is because all we need are pseudo random numbers, numbers which have to correlation to what we are testing and provide some statistical randomness. One way to check this is repeatability of results, if there is a bias the results will not produce similar outcomes for a large enough sample size.


In [31]:
# Defining the number of random numbers to be generated, how many bins they go into and how many times this is run
number_of_numbers = 1000
number_of_bins = 10
number_of_trials = 10

# Creating the figure
fig = plt.figure()
ax = fig.add_subplot(111)

# Creating an empty array to store all bin counts for each trial
random_nums_counts = np.empty((0, number_of_bins))

# Generating the random numbers and plotting the corresponding histograms for however many trials are specified
for _ in range(number_of_trials):
    random_nums_test = np.random.uniform(size=number_of_numbers)
    count, bin_edges, _ = ax.hist(random_nums_test, number_of_bins, edgecolor='k', alpha=0.2)
    # The dummy variable stores texture information that we don't need

    # Storing the bin counts for this run
    random_nums_counts = np.vstack((random_nums_counts, count))

# Finding the mean count and the error on each bin
count_means = np.mean(random_nums_counts, axis=0)
count_errs = np.std(random_nums_counts, axis=0)

# Plotting these mean counts and errors
ax.bar(bin_edges[:-1], count_means, align='edge', edgecolor='k', alpha=0.5, width=1/number_of_bins, color='green',
       yerr=count_errs, label='calculated means')
# bin_edge stores the left edge of each bar including a 'n+1'th bar so we remove the last element

# Defining a label for the error-bars
black_line = mlines.Line2D([], [], color='black', marker='|', label='calculated errors')

# Finding the expected mean and plotting it
expected_value = number_of_numbers / number_of_bins
ax.axhline(expected_value, color='r', alpha=0.8, label='predicted mean')

# Finding the expected error and plotting it
expected_error = np.sqrt(expected_value * (number_of_numbers - expected_value) / number_of_numbers)
ax.axhline(expected_value + expected_error, color='r', linestyle='--', alpha=0.7, label='predicted error')
ax.axhline(expected_value - expected_error, color='r', linestyle='--', alpha=0.7)

# Labelling the plot
ax.set_xlabel('x')
ax.set_ylabel('count')
ax.set_title('Uniform random numbers in 1D')

# Displaying the legend and the plot
handles, labels = ax.get_legend_handles_labels()
handles.append(black_line)
plt.legend(loc='lower right', handles=handles)
plt.show()

<IPython.core.display.Javascript object>

From this we can see that all averages are within the error range and all errors on these average counts are visually within 2 standard deviations. Additionally, the error-bars are roughly the correct sizes and are both consistent with each other and the predicted value. So we can conclude inbuilt numpy function does indeed produce a uniformly distributed set of random numbers.

To get better results we would have to include more samples but this would come at the cost of computational time which while miniscule now, will be a massive factor later on. Since even for 10 trials the averages are all well within the error, we can use this as a good middle-ground between precision and time and so this will be our standard from now on.

To properly conclude the optimal value, many different numbers of trials would have to be compared but this is not done here as it would be of little benefit overall as there is only so much we can conclude from a 1D sample and this would not be worth the time.

## Generating uniform random numbers in 3D

Now that we have established the uniformity of the random numbers, it is always worth plotting them in 3D to visually check for any bias or any other problems. An example of a potential problem could be what is known as the 'spectral problem' which is where the random number generator produces values that when visualised in 3D lie on hyperplanes (Marsaglias' Theorem). It essentially means that successive points are correlated. This is something that is only visible in 3 dimensions, or higher, hence our use of a 3D plot.

Whilst there exist various generators that are known to have this problem, such as an LCG (Linear Congruential Generator), we will use another method, RANDSSP (Multiplicative congruential uniform random number generator), to show this. RANDSSP is based on the parameters used by IBM's Scientific Subroutine Package and was used in several libraries in the 1960's and displays strong serial correlation between three consecutive values. So to easily visualise the problem, we assign the 3 consecutive random numbers as the x, y and z coordinated of a given point as this will naturally cause the final spacial distribution of many points to contain a hidden bias. This is done as opposed to say assigning the 3 consecutive random numbers as consecutive x values as correlation between the 3 particles would be hard to visualise.

In [32]:
# Defining a function to generate random numbers according to RANDSSP, numbers that exhibit the spectral problem. The function was taken from the demonstrator.
# The statement : r = randssp(m,n), generates an m-by-n random matrix.
# The function cannot accept any other starting seed.

def randssp(p, q):

    global m, a, c, x

    try: x
    except NameError:
        m = pow(2, 31)
        a = pow(2, 16) + 3
        c = 0
        x = 123456789

    try: p
    except NameError:
        p = 1
    try: q
    except NameError:
        q = p

    r = np.zeros([p,q])

    for l in range (0, q):
        for k in range (0, p):
            x = np.mod(a*x + c, m)
            r[k, l] = x/m

    return r

In [33]:
# Defining a function to produce 2 3D plots given coordinates in 2D arrays of the form (3, N) for N points. Arrays need to be in
# this orientation to easily plot results and compare with RANDSSP
def double_scatter_plot_3d(array_3d_set1, array_3d_set2, title_1, title_2, overall_title):

    array_3d_sets = [array_3d_set1, array_3d_set2]
    titles = [title_1, title_2]

    # Initialising the figures and subplots
    fig, axes = plt.subplots(1, len(array_3d_sets), figsize=(10, 6), subplot_kw={'projection': '3d'})

    # Adding an overarching title
    fig.suptitle(overall_title, y=0.97)

    # Looping through both sets of 3d random numbers to plot each one
    for index, set_3d in enumerate(array_3d_sets):
        # Defining the current subplot
        ax = axes[index]

        # Plotting the points from the array using slices
        ax.scatter(set_3d[0, :], set_3d[1, :], set_3d[2, :], marker='.')

        # Labelling the plot
        ax.set_xlabel('x')
        ax.set_ylabel('y')
        ax.set_zlabel('z')
        ax.set_title(titles[index])

        # Changing the visual size of the axis to make them all uniform and nicely orientated
        ax.set_box_aspect(aspect = (1,1,1))
        ax.view_init(azim=60, elev=30)

    # Displaying the plot
    plt.tight_layout()
    plt.show()

# Generating a 2d array of uniform random numbers with x,y,z down the rows and N=1000 values along the columns
uniform_nums_3d = np.random.uniform(size=(3, number_of_numbers))

# Calling the function to generate a set of random numbers with the spectral problem
problem_nums_3d = randssp(3, number_of_numbers)

# Calling the function to plot the 2 sets of random numbers
double_scatter_plot_3d(uniform_nums_3d, problem_nums_3d, 'Uniform numpy', 'RANDSSP', 'Random numbers scatter plot in 3D')

<IPython.core.display.Javascript object>

From this we can see that the random numbers generated using RANDSSP do indeed lie on hyperplanes failing the spectral test while the random numbers from the uniform distribution pass this test. Additionally, these numbers don't seem to visually have any other hidden biases and so we can conclude that they do indeed, atleast for out purposes, exhibit statistical randomness.

## Simulating one process in 1D

Now that we have a random number generator, we can simulate a simplified process to the one we are eventually going to study which is a good way to test our theory. Here we look at the 1D case with no scattering for which the aforementioned theory still holds.

What we are going to do is put the nominal value for the mean free path and see if the outputted value ,calculated using the intensity variation with distance, is the same. To do this we now need to produce random numbers according to the exponential distribution as this is what the equation derived earlier showed, as distance into the material increases the number of neutrons decreases exponentially. The constant $I_{0}$ is irrelevant here as we only want the overall behaviour.

The method used here is inverse/cumulative distribution sampling. This starts by defining the required probability density function (pdf) which for our exponential distribution:
$$ {pdf}(x) = e^{-\frac{x}{\lambda_{a}}} $$
where $\lambda_{a}$ is as before the mean free path for absorption. We then find the corresponding cumulative density function (cdf) by integrating:
$$ {cdf}(x) = \int_{0}^{x} {pdf}(x) dx = \lambda_{a}(1- e^{-\frac{x}{\lambda_{a}}}) $$
but since we are just interested in the functional form and ${cdf}(x) = 1$ must be true as $x \rightarrow \infty$, we drop the $\lambda_{a}$ pre-factor. Inverting this we get the final sampling function as:
$$ {cdf}^{-1}(x) = \lambda_{a}ln(1-x). $$
However, since $x$ is just a random number between $0$ and $1$, $1-x$ is still just a random number over the same range and so we can simply this expression as:
$$ s_{i} = -\lambda_{a}ln(u_{i}) $$
where $s_{i}$ is a random number that follows the exponential distribution and $u_{i}$ is a uniform random number and $\lambda_{a}$ is the expected mean free path of absorption.

In [34]:
# Defining a function to produce random numbers in an exponential distribution according to the theory above
def random_num_exponential_dist(size, mean_free_path):

    random_nums_1d = np.random.uniform(size=size)

    return - mean_free_path * np.log(random_nums_1d)

Using this we can produce set of random numbers and plot a histogram to show that these numbers do indeed follow an exponential distribution. As before to get a better estimate, we do mutilple trials and take an average and also find the error on the counts on each bin this way.

To actually extract a $\lambda_{a}$ from all this sampling, we rearrange the intensity decay equation as:
$$ ln(N_{R}) = -\frac{R_{i}}{\lambda_{a}} $$
where $N_{R}$ is the number of neutrons (count in each bin) and $R_{i}$ is the distance into the material. This is done so that we can plot a straight line where:
$$ gradient = -\frac{1}{\lambda_{a}} $$
allowing us to extract a value for $\lambda_{a}$.

Because we have multiple trials, there will be an error on $N_{R}$ which when propagated through, gives an error on the gradient and thus an error on $\lambda_{a}$ which can be found from:
$$ \sigma_{grad} = \frac{\sigma_{\lambda_{a}}}{\lambda_{a}^2} $$
where $\sigma_{grad}$ and $\sigma_{\lambda_{a}}$ are the errors on the gradient and mean free path respectively (and should not be confused with microscopic cross-sections values which use the same symbol).

In [35]:
# Defining a function to fit a lambda value to the data
def fit_lambda(x_vals, ln_y_vals, ln_y_errs):

    # Fitting the best fit line to the data and finding the corresponding fir parameters and covariance matrix
    ln_y_errs_weights = 1 / ln_y_errs
    params, cov = np.polyfit(x_vals, ln_y_vals, deg=1, w=ln_y_errs_weights, cov=True)

    # Extracting the gradient and intercept
    gradient = params[0]
    intercept = params[1]

    # Finding the mean free path from the gradient
    lambda_val = - 1 / gradient
    # Finding the error on the gradient and error on the mean free path
    gradient_err = np.sqrt(cov[0][0])
    lambda_err = gradient_err / gradient**2

    # Finding the points corresponding to the line of best fit
    fit_points = x_vals * gradient + intercept

    return fit_points, lambda_val, lambda_err

# Defining a function to numerically find the mean free path of absorption for a given material
def find_lambda_absorption_1d(material, number_of_neutrons, trials):

    # Finding the expected lambda value while only considering absorption processes
    lambda_absorption = neutron_data[material]['lambda absorption']
    # Correcting the units for lambda to the required units of cm
    lambda_absorption = lambda_absorption * 10**24

    # Initialising the figure and first subplot
    fig = plt.figure(figsize=(9, 4))
    ax_1 = fig.add_subplot(121)

    fig.suptitle(f'{material.capitalize()} in the absence of scattering')

    # Since each set of random numbers from the exponential distribution has a variable range, if we assign a set number of bins
    # the width of each bin will be inconsistent which makes further analysis based on counts in each bin pointless. So we
    # instead set a fixed bin width to fix this issue.
    bin_width = 10
    bin_no_max = 100
    # The number of possible bins is in theory infinity but to make the code work (specifically np.vstack) a set number of bins
    # is needed and 100 is a good enough number since the probability of a neutron travelling over 50 times it's mean free path
    # without colliding is practically 0 (2.2x10^-10 = 2.2x10^-6 for 10x1000=10,000 neutrons)

    # Creating an empty array to store all bin counts for each trial
    exp_nums_counts = np.empty((0, bin_no_max))

    # Creating an empty list to store the maximum distances travelled in each trial
    max_distances = []

    # Running the procedure a set number of times
    for _ in range(trials):
        # Finding the array of random numbers according to an exponential distribution
        random_nums_exp = random_num_exponential_dist(number_of_neutrons, lambda_absorption)

        # Finding the maximum distance travelled and appending it to the list
        max_distance= np.max(random_nums_exp)
        max_distances.append(max_distance)

        # Re-doing the loop in the unlikely event the neutron travelled 1000cm so as to not break the code and printing an error message to let us know
        distance_threshold = bin_no_max * bin_width
        if max_distance > distance_threshold:
            print(f"A neutron travelled over {distance_threshold}m and was omitted as an outlier.\n")
            continue

        # Defining the position of the bins based on the maximum distance travelled
        bin_edges = np.arange(0, max_distance + bin_width, bin_width)

        # Plotting the histogram and getting the counts in each bin
        count, _, _ = ax_1.hist(random_nums_exp, bin_edges, edgecolor='k', alpha=0.2)

        # Resizing the count array and storing them for this run
        count = count.copy()
        # This is to prevent a ValueError (which is to do with OWNDATA=False which means the array is a view of another array)
        count.resize((bin_no_max,), refcheck=False)
        # If the shape is bigger than the length of counts, the rest of the numbers are filled with zeros
        exp_nums_counts = np.vstack((exp_nums_counts, count))

    # Removing any trialing zeros maximum of all maximum distances
    bin_edges = np.arange(0, np.max(max_distances), bin_width)
    exp_nums_counts = exp_nums_counts[:, :len(bin_edges)]

    # Finding the mean count and the error on each bin
    exp_count_means = np.mean(exp_nums_counts, axis=0)
    exp_count_errs = np.std(exp_nums_counts, axis=0)

    # Plotting these mean counts and errors
    ax_1.bar(bin_edges, exp_count_means, align='edge', edgecolor='k', alpha=0.5, width=10, color='green',
             yerr=exp_count_errs, label='mean counts')

    # Defining a label for the error-bars
    black_line = mlines.Line2D([], [], color='black', marker='|', label='count errors')

    # Finding the bins with zero counts and removing those points to not get ln(0) errors
    zeros_indices = np.where(exp_count_means == 0)[0]
    exp_count_means = np.delete(exp_count_means, zeros_indices)
    exp_count_errs = np.delete(exp_count_errs, zeros_indices)
    bin_edges = np.delete(bin_edges, zeros_indices)

    # Changing the bin points from left edge to centre for better plotting
    bin_half_width = bin_width / 2
    bin_centres = bin_edges + bin_half_width

    # Finding the natural log of the counts and the error on it
    ln_counts = np.log(exp_count_means)
    ln_counts_err = exp_count_errs / exp_count_means

    # Initialising the second subplot and plotting ln(counts) against bin_centres to give a straight line
    ax_2 = fig.add_subplot(122)
    ax_2.errorbar(bin_centres, ln_counts, yerr=ln_counts_err, xerr=bin_half_width, fmt='x')

    # Calling the function to find the fitted lambda value and the error
    best_fit_points, lambda_absorption_found, lambda_absorption_err = fit_lambda(bin_centres, ln_counts, ln_counts_err)

    ax_2.plot(bin_centres, best_fit_points, color='r', label='best-fit line')

    # Labelling both the subplots
    ax_1.set_xlabel('R_i')
    ax_1.set_ylabel('N_i')
    ax_1.set_title('Exponentially distributed random numbers')

    ax_2.set_xlabel('R_i')
    ax_2.set_ylabel('ln(N_i)')
    ax_2.set_title('Finding attenuation length')

    # Displaying the legend and the plot
    handles, labels = ax_1.get_legend_handles_labels()
    handles.append(black_line)
    ax_1.legend(loc='upper right', handles=handles)
    ax_2.legend(loc='upper right')
    plt.tight_layout()
    plt.show()

    # Printing all the results
    print(f'The characteristic attenuation length of {material} in the absence of scattering was predicted to be '
          f'{lambda_absorption:.2f} cm and is found to be {lambda_absorption_found:.2f} ± {lambda_absorption_err:.2f} cm')

For this test we consider water as our material as it has the highest absorption cross-section of all 3 materials so it should give the easiest to visualise results with the lowest mean free path of absorption.

In [36]:
%%time

# Calling the function to find the value for water for N=1000 neutrons and 10 trials
find_lambda_absorption_1d('water', number_of_numbers, number_of_trials)

<IPython.core.display.Javascript object>

The characteristic attenuation length of water in the absence of scattering was predicted to be 44.98 cm and is found to be 44.98 ± 0.38 cm
CPU times: total: 297 ms
Wall time: 292 ms


From this we can see that the value found is usually either within or almost within one standard deviation of the true expected value which overall indicates that this methodology works and the theory is sound. The best fit line is also visually a good fit which further backs supports this. The reason for the deviation despite initially inputting the true value is because of randomness in our exponential distribution because of our use of random numbers. To get perfect agreement an infinite number of trials would have to be done but this obviously impractical and so as before, 10 trials is good enough for our purposes. Additionally, since this result is so close to the true value, there is further merit to using this 10 trial number as our sweet-spot.

We are now ready to expand this approach to 3D and include both processes, scattering and absorption.

## Generating non-uniform numbers in 3D

To simulate neutron motion in 3D, we need to expand our approach of using exponentially distributed random numbers from 1D to 3D and the way we do this is in 2 steps. First we produce isotropic unit vectors, and then we scale them using the exponential distribution.

As mentioned we start with producing isotropic unit vectors which are uniformly distributed random points on the surface of a sphere. The reason these need to be isotropic is because the direction a neutron will take after colliding is assumed to be random for thermal neutrons. The way this is done is by exploiting spherical symmetry to transform random numbers in $r$, $\theta$, $\phi$ spherical polar coordinates into $x$, $y$, $z$ cartesian coordinates using:
$$ x_{i} = r sin(\theta_{i})cos(\phi_{i}) $$
$$ y_{i} = r sin(\theta_{i})sin(\phi_{i}) $$
$$ z_{i} = r cos(\theta_{i}) $$
where $r$ denotes the length of the step taken by the neutron but for now we take $r = 1$ as we want unit vectors.

For our range of supplied $\theta_{i}$ and $\phi_{i}$ values, we chose the same limits as when integrating over a sphere, namely $0 < \theta_{i} < \pi$ and $0 < \phi_{i} < 2\pi$, but we have to be careful when sampling from them. While we can keep the $\phi$ values uniform, if we keep the $\theta_{i}$ values uniform as well we will end up with more points near the poles and not our desired uniform distribution. This is because as you go from the equator to the poles, the cross-section of the sphere decreases and by using a uniform theta sample we would be trying to distribute points evenly over these uneven cross-sections which would undoubtedly lead the smaller cross-sections, the poles, having more points giving an unwanted bias.

To fix this we need to sample theta values in such a way as to generate more values near the equator and less near the poles. This can be done by distributing them according to:
$$ {pdf}(\theta_{i}) = \frac{1}{2} sin(\theta_{i}) $$
which by the inverse distribution sampling method can be achieved by having:
$$ \theta_{i} = arccos(1-2u_{i}) $$
where $u_{i}$ is a uniformly distributed random number.

As always it is important to visually check whether or not our theory holds and to see if what we did fixed the postulated problem if there are any other problems/biases that we need to fix.

In [37]:
# Defining a function to produce random points on a unit sphere (isotropic unit vectors)
def isotropic_vectors(size, radius=None, bias=False):

    # Assigning initial random theta and phi arrays
    random_theta = np.random.uniform(size=size)
    random_phi = np.random.uniform(size=size)

    # Correcting the limits and distributions of the phi array
    random_phi = random_phi * 2 * np.pi

    # Correcting the limits of and distributions of the theta array depending on if a biased sample is wanted or not
    if bias:
        random_theta = random_theta * np.pi
    else:
        random_theta = np.arccos(1 - 2 * random_theta)

    # Assigning a unit radius if none is explicitly provided
    if radius is None:
        radius = np.ones(size)

    # Finding the coordinated of each random point according to the equations above
    random_x = radius * np.sin(random_theta) * np.cos(random_phi)
    random_y = radius * np.sin(random_theta) * np.sin(random_phi)
    random_z = radius * np.cos(random_theta)

    # Formatting the output array to match previous 3d random point arrays for convenience
    output_array = np.array([random_x, random_y, random_z])

    return output_array

# Redefining the number of points being sampled to better view the bias
number_of_numbers = 2000

# Calling the function to generate some biased random isotropic unit vectors not uniformly distributed
iso_unit_biased = isotropic_vectors(2000, bias=True)

# Calling the function to generate some random isotropic unit vectors uniformly distributed
iso_unit_uniform = isotropic_vectors(2000)

# Calling the function defined previously to plot the 2 sets of random numbers
double_scatter_plot_3d(iso_unit_biased, iso_unit_uniform, 'Uniform theta values', 'Non-uniform theta values', 'Isotropic unit vectors in 3D')
# scatter_plot_3d(random_iso_unit_trial, title='Random isotropic unit vectors')

<IPython.core.display.Javascript object>

As expected, when uniform theta values are used there is a bias giving more points near the poles but when a non-uniform sample is used this is corrected. Visually there appear to be mo more biases and so we can move onto the second step.

The second step, as mentioned, involves scaling the isotropic unit vectors. To do this we simply obtain the distance the neutron would move in the exact same way as we did in the 1D case, using the exponential distribution, and input this value as the length of the step, $r$, in the earlier equation. This scales the isotropic unit vectors so they are exponentially distributed isotropic vectors.

To visually check if this works, we produce not only a scatter plot but also a histogram to see how the number of neutrons scales with distance of the neutron from centre (found using Pythagoras' Theorem). If our theory holds, this should recover an exponential distribution and so serves as a good visual check. Furthermore, to easily check our results we have chosen to use the same value of lambda as we did for the 1D case of absorption, namely the mean free path of absorption for neutrons in water.

In [38]:
# Defining a function to generate exponentially distributed isotropic unit vectors
def isotropic_exponential_vectors_random(size, mean_free_path):
    # Lambda should already be in the correct units (cm)

    # Generating the exponentially distributed random points
    exponential = random_num_exponential_dist(size, mean_free_path)

    # Generating the isotropic exponential random points
    isotropic_exponential = isotropic_vectors(size, radius=exponential)

    return isotropic_exponential

# Defining a function to plot the isotropic exponential random points
def scatter_histogram_plot(num_array):

    # Initialising the figure and subplots
    fig = plt.figure(figsize=(10, 6))
    ax_1 = fig.add_subplot(121, projection='3d')
    ax_2 = fig.add_subplot(122)

    # Adding an overarching title
    fig.suptitle('Random isotropic exponentially distributed points', y=0.97)

    # Finding the x, y and z values of each point
    x_vals = num_array[0, :]
    y_vals = num_array[1, :]
    z_vals = num_array[2, :]

    # Finding the distance from the center of each point
    distance = np.sqrt(x_vals**2 + y_vals**2 + z_vals**2)

    # Plotting the points on the first subplot with a heatmap that scales with distance from the centre
    ax_1.scatter(x_vals, y_vals, z_vals, marker='.', c=distance, cmap='viridis')

    # Labelling the first subplot
    ax_1.set_xlabel('x')
    ax_1.set_ylabel('y')
    ax_1.set_zlabel('z')
    ax_1.set_title('Scatter plot')

    # Changing the visual size of the axis to make them all uniform and nicely orientated
    ax_1.set_box_aspect(aspect = (1,1,1))
    ax_1.view_init(azim=60, elev=30)

    # Plotting the variation of neutron counts with distance in the second subplot
    _, bin_left_edges, _ = ax_2.hist(distance, bins=20, edgecolor='k', color='green')

    # Labelling the second subplot
    ax_2.set_xlabel('Distance from the centre')
    ax_2.set_ylabel('Count')
    ax_2.set_title('Histogram')

    # Displaying the plot
    fig.tight_layout()
    plt.show()

# Finding a suitable value of lambda to trial
lambda_water_absorption = neutron_data['water']['lambda absorption']
# Correcting the units for lambda to the required units of cm
lambda_water_absorption = lambda_water_absorption * 10**24

# Calling the function to generate the isotropic exponential random points
iso_exp_trial = isotropic_exponential_vectors_random(number_of_numbers, lambda_water_absorption)

# Calling the function to plot these random points
scatter_histogram_plot(iso_exp_trial)

<IPython.core.display.Javascript object>

From this we can see that our generation method works as intended and we are able to generate isotropic random points which follow an exponential distribution. The histogram, while this time not having any repeats, is similar in shape to the 1D histogram produced earlier and displays that the count falls what looks to be exponentially as you go out from the center which is exactly what we need.

It should be noted that the reason we are only interested in a visual check and do need multiple repeats is because we already had an in-depth look into each process, isotropic unit vector generation and exponential distribution generation, and determined that they work fine. This is just a simple extension to those ideas and so all we need to do is essentially a sanity check to make sure nothing has gone wrong when combining the methods!

## Simulating multiple processes in 3D

Now that we have a solid understanding of random number generation and neutron absorption in 1D, we can return to the main objective of this report. Other than now looking in 3D, the first thing we have to consider is what happens if we now have 2 processes, absorption and scattering, as opposed to just the one as we have been dealing with so far.

As mentioned earlier, all the equations derived for absorption are the same for scattering, the only thing that changes is the inputted microscopic cross-section. We can use this to actually combine both absorption and scattering processes to get a total macroscopic cross-section given by:
$$ \Sigma_{T} = \Sigma_{a} + \Sigma_{s} $$
where the subscripts $_{a}$ and $_{s}$ denote absorption and scattering respectively as before. We can also get a total mean free path given by:
$$ \lambda_{T} = \frac{1}{\Sigma_{T}} = \frac{1}{\frac{1}{\lambda_{a}} + \frac{1}{\lambda_{s}}} $$
where this value now denotes the mean free path of a neutron before it collides with any type of particle in the material (not just an absorbing particle) and is overall a much more realistic view of our system.

Now that we have this, we still need some way to differentiate absorbing particles and scattering processes. We do this by assigning a probability of absorption and a probability of scattering as:
$$ P_{i} = \frac{\Sigma_{i}}{\Sigma_{T}} $$
where $i$ is a placeholder for either $a$ or $s$ depending on the probability we need.

It should also be mentioned that we can use this $\lambda_{T}$ as before to generate exponentially distributed random numbers as:
$$ s_{i} = \lambda_{T}ln(u_{i}) $$
as we had previously and this is what we will be using from now on when sampling from an exponential distribution.

In [39]:
# Defining a function to calculate and store the total mean free path using the equation above
def find_total_mean_free_path(material):
    macro_absorb = neutron_data[material]['absorption macro']
    macro_scatter = neutron_data[material]['scattering macro']

    macro_total = macro_scatter + macro_absorb
    neutron_data[material]['total macro'] = macro_total

    try:
        lambda_combined = 1 / macro_total
    except ZeroDivisionError:
        lambda_combined = np.inf
    neutron_data[material]['lambda total'] = lambda_combined

# Defining a function to calculate and store the absorbing and scattering probabilities
def find_probabilities(material):
    cross_section_macro_absorb = neutron_data[material]['absorption macro']
    cross_section_macro_scatter = neutron_data[material]['scattering macro']
    cross_section_macro_total = cross_section_macro_absorb + cross_section_macro_scatter

    try:
        absorption_probability = cross_section_macro_absorb / cross_section_macro_total
        scattering_probability = cross_section_macro_scatter / cross_section_macro_total
    except ZeroDivisionError:
        absorption_probability = 0
        scattering_probability = 0

    neutron_data[material]['scattering probability'] = scattering_probability
    neutron_data[material]['absorption probability'] = absorption_probability

# Looping through each material
for test_material in material_list:
    # Calling the two functions above to find the parameters for each material
    find_total_mean_free_path(test_material)
    find_probabilities(test_material)

We now have all that we need to simulate the motion of a neutron through a material! What we are interested in is how many neutrons are reflected, absorbed or transmitted and so we have to keep a count of each. Other things like the full particle history is not necessary for every neutron as there are simply too many and there is no physical use in keeping track of what is essentially random motion. Nevertheless, we will still keep track of some histories so that we can plot them later on but this properly is discussed a little later.

If we look at a neutron's cycle, we can see how it's random walk evolves using a step-by-step procedure that loops until the neutron's history has concluded or there are no more neutrons left to sample:

1) We chose a step for the neutrons to take by sampling from to the random isotropic exponential distribution and move the neutrons according to this step. It should be noted that since we are looking at neutrons normally incident on a material, the first step is taken to be in the $x$ direction and so for that step we simpy sample from the 1D exponential distribution.

2) We check to see if the neutron is still within the material. If it has been reflected we take a note of it and add to a transmission tally and similarly for if it has been transmitted. In both cases we kill the neutron history as it is no longer within our region of interest.

3) If the neutron is still within the material, we check to see if it has been absorbed or not. This is done using the Von Neumann Rejection Method where we generate a uniform random number between 0 and 1; if this number is less than or equal to $P_{a}$ we take the neutron to be absorbed otherwise we take it to be scattered. As before, if the neutron is absorbed, we add to the absorption tally and kill its history.

What this finally gives is a tally for the number of neutrons reflected, absorbed and transmitted which is what we need. To speed up this process we simulate all the neutrons at once since using numpy arrays is much faster than looping though each neutron one at a time. This vastly reduces computational time, for even as low as 100 sampled neutrons, and allows for us to run this many many times to get errors on these values and investigate how the values change with material, thickness and numbers of neutrons. This is explored a little later.

It should be stated that we are considering each neutron's motion independently and so are not allowing for neutron-neutron collision. While this is much less likely than a neutron's interaction with a particle in the material, it is still possible and the probability for it only becomes higher for more neutrons being sampled. This is dome for simplicity in simulation and can be considered to be a limitation of our model although in practicality the results from this model still serve as a good estimate.

In [40]:
# Define a function to model the random walk for a given number of neutrons for a given material of a given thickness
def random_walk(material, thickness, number_of_neutrons, history=False):

    # Defining the initial number of neutrons to comapre to late
    initial_number = number_of_neutrons

    # Finding the total mean free path value and correcting the units to cm
    lambda_total = neutron_data[material]['lambda total']
    lambda_total = lambda_total * 10**24

    # Finding the probability of absorption
    probability_absorbed = neutron_data[material]['absorption probability']

    # Defining the initial count tallies
    total_absorbed = 0
    total_reflected = 0
    total_transmitted = 0

    # Generating the first step directly
    neutron_positions = np.zeros((2, number_of_neutrons))
    first_step_exp = random_num_exponential_dist(number_of_neutrons, lambda_total)
    neutron_positions = np.vstack((first_step_exp, neutron_positions))
    # Ordering the arrays like this in vstack ensures the x values remain on-top as needed

    # Creating an array to store position history if asked for
    if history:
        neutron_position_history = np.zeros((3, 1))
    # This only really works as intended if the number of neutrons being sampled is 1 and that is all it is used for

    # Repeating the random walk cycle unitl all neutron histories are killed
    while number_of_neutrons > 0:

        # Adding the new position data for the neutron history if asked for
        if history:
            neutron_position_history = np.hstack((neutron_position_history, neutron_positions))

        # Finding which neutrons have been reflected
        reflected_indices = np.where(neutron_positions[0] < 0)[0]
        # Adding to the reflection tally
        number_reflected = len(reflected_indices)
        total_reflected = total_reflected + number_reflected
        # Deleting the relevant neutrons from the array
        neutron_positions = np.delete(neutron_positions, reflected_indices, axis=1)

        # Finding which neutrons have been transmitted
        transmitted_indices = np.where(neutron_positions[0] > thickness)[0]
        # Adding to the transmission tally
        number_transmitted = len(transmitted_indices)
        total_transmitted = total_transmitted + number_transmitted
        # Deleting the relevant neutrons from the array
        neutron_positions = np.delete(neutron_positions, transmitted_indices, axis=1)

        # Redefining the number of neutrons now left still within the material
        number_of_neutrons = number_of_neutrons - number_reflected - number_transmitted

        # Generating uniform random number to test which neutrons have been absorbed
        probability_random_nums = np.random.uniform(size=number_of_neutrons)
        absorbed_indices = np.where(probability_random_nums <= probability_absorbed)[0]
         # Adding to the absorption tally
        number_absorbed = len(absorbed_indices)
        total_absorbed = total_absorbed + number_absorbed
        # Deleting the relevant neutrons from the array
        neutron_positions = np.delete(neutron_positions, absorbed_indices, axis=1)

        # Redefining the number of neutrons now left still within the material
        number_of_neutrons = number_of_neutrons - number_absorbed

        # Generating the next step for all neutrons sill within the material
        random_points = isotropic_exponential_vectors_random(number_of_neutrons, lambda_total)
        neutron_positions = neutron_positions + random_points

    # Defining a list of value to be returned
    return_list = [total_reflected, total_absorbed, total_transmitted]

    # Compares the total of the 3 tallies to the initial number of neutrons, if there is a mismatch we print a warning.
    total_number = np.sum(return_list)
    if total_number != initial_number:
        print(f"The initial number of neutrons was taken to be {initial_number} while the final number has come out as {total_number}. The total number of neutrons has not been conserved for this run of {thickness}cm {material}.\n")

    # Adding neutron history to that list if asked for
    if history:
        return_list.append(neutron_position_history)

    return return_list

As alluded to earlier, it is sometimes important to keep track of particle histories particularly when it comes to plotting the random walk of a neutron. The reason we want to do this before performing any other tests is to both visually see if our random walk looks reasonable to give some validation to our method but also to just observe how a neutron may travel in a material.

So as to not clutter the plot, we only plot one of each type of event that can happen. In other words we plot one instance where the neutron is reflected, one where it is transmitted and one where it is absorbed. Since this is a random process this can sometimes lead to very short neutron paths but this is an aspect of random sampling and so we ignore it. To give a better visual representation we also plot vertical planes to signify material boundaries which also helps with give visual orientation to the plot. This is done for all 3 materials here so we can compare them.

We should note that another 'limitation' of this model is the fact that we are considering a material with infinite height and width. This effectively amounts to saying that the material is much bigger than the mean free path of the neutron which is a reasonable approximation to make in say a nuclear reactor. It is for this reason that this is less of a limitation and more of a design choice, albeit one to make the simulation easier.

In [41]:
# Listing the different types of outcomes and colors corresponding to each in a specific according to how they were returned by
# the random_walk function. This order is maintained throughout.
outcome_types = ['reflected', 'absorbed', 'transmitted']
outcome_colours = ['tab:blue', 'tab:red', 'tab:green']

# Defining a function to plot the random walk of a particle in a given material of a given thickness
def plot_random_walk(material, thickness, method='standard'):

    # Initialising the figure
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    # Setting up some flags to see if this outcome still needs to be plotted
    reflected_flag = True
    absorbed_flag = True
    transmitted_flag = True

    # Repeating the sampling until all the processes have been displayed
    while reflected_flag or absorbed_flag or transmitted_flag:
        # Calling the function to simulate one neutron's random walk including it's history
        if method == 'standard':
            reflected, absorbed, transmitted, neutron_history = random_walk(material, thickness, 1, history=True)
        elif method == 'woodcock':
            reflected, absorbed, transmitted, neutron_history = woodcock_random_walk(material, thickness, 1, history=True)

        # Checking to see what outcome the neuron ends up with and plotting its path if the outcome has not already been plotted.
        if reflected == 1 and reflected_flag:
            ax.plot(neutron_history[0, :], neutron_history[1, :], neutron_history[2, :], marker = '.', color=outcome_colours[0],
                    label=outcome_types[0])
            reflected_flag = False

        elif absorbed == 1 and absorbed_flag:
            ax.plot(neutron_history[0, :], neutron_history[1, :], neutron_history[2, :], marker='.', color=outcome_colours[1],
                    label=outcome_types[1])
            absorbed_flag = False

        elif transmitted == 1 and transmitted_flag:
            ax.plot(neutron_history[0, :], neutron_history[1, :], neutron_history[2, :], marker='.', color=outcome_colours[2],
                    label=outcome_types[2])
            transmitted_flag = False

        # In theory this check could be done in a for loop since the check is fairly repetitive but this is about the same lines of code, about the same computationally but is much easier to read so we stick with this.

    # Labelling the plot and giving it a title
    ax.set_xlabel('x (cm)')
    ax.set_ylabel('y (cm)')
    ax.set_zlabel('z (cm)')
    if method == 'standard':
        ax.set_title(f'Three neutron histories for {thickness}cm {material}')
    elif method == 'woodcock':
        ax.set_title(f'Three neutron histories for {thickness[0]}cm {material[0]} and {thickness[1]}cm {material[1]}')

    # Finding the minimum and maximum y and z limits of the axes
    y_lims = ax.get_ylim()
    z_lims = ax.get_zlim()
    y_min, y_max = y_lims[0], y_lims[1]
    z_min, z_max = z_lims[0], z_lims[1]

    # Creating a yz mesh to plot create a surface plot to indicate material boundaries
    y_vals = np.linspace(y_min, y_max, 100)
    z_vals = np.linspace(z_min, z_max, 100)
    y_mesh_z, z_mesh_y = np.meshgrid(y_vals, z_vals)

    # Plotting the surface plot using the mesh grid at the two boundaries
    ax.plot_surface(0, y_mesh_z, z_mesh_y, alpha=0.5, color='k')
    if method == 'standard':
        ax.plot_surface(thickness, y_mesh_z, z_mesh_y, alpha=0.5, color='k')
    elif method == 'woodcock':
        ax.plot_surface(thickness[0], y_mesh_z, z_mesh_y, alpha=0.5, color='k')
        ax.plot_surface(np.sum(thickness), y_mesh_z, z_mesh_y, alpha=0.5, color='k')

    # Adding the surface plot to the legend
    black_patch = mpatches.Patch(color='k', label='material edge', alpha=0.5)
    handles, labels = ax.get_legend_handles_labels()
    handles.append(black_patch)

    # Changing the visual size of the axis to make them all uniform and nicely orientated
    ax.set_box_aspect(aspect = (1,1,1))
    ax.view_init(azim=120, elev=25)

    # Drawing two dotted lines to show where the 0,0,0 point of entry is
    zero_y_line = np.linspace(y_min, y_max, num=10)
    zero_z_line = np.linspace(z_min, z_max, num=10)
    zero_line = np.zeros((10,))
    ax.plot3D(zero_line, zero_y_line, zero_line, color='black', ls='dashed', alpha=0.5)
    ax.plot3D(zero_line, zero_line, zero_z_line, color='black', ls='dashed', alpha=0.5)

    # Displaying the legend and the plot
    plt.legend(handles=handles)
    plt.show()


In [42]:
%%time

# Assigning an initial material thickness
initial_thickness = 10

# Looping through each material and calling the function to plot random walks for each one
for test_material in material_list:
    plot_random_walk(test_material, 10)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

CPU times: total: 1.84 s
Wall time: 1.81 s


These plots allow us to see that our random walk works as intended. There appears to be no correlation in the direction the neutron travels, the step-size it takes is non-uniform and any neutron that exits the bounds of the material is immediately killed all as we want. It is important to note that all neutrons start at the same point but travel in random directions so the relative start point between different graphs is different depending on their axis. This could be a cause for confusion so dotted have been plotted so that their intersection indicates the point where the neutrons are incident on the material.

As far as looking at variation between different materials, water has the most steps taken as it has by far the shortest mean free path which yields some nice variation in the plots. Lead and Graphite on the other hand are very comparable to one another as their mean free paths are comparable to the thickness of the material. From this we could say that water has more accurate results for this thickness since having so many more steps reduces the impact of a potential huge step-size from the exponential distribution sample but we would need more tests that specifically study this to conclude anything, something which is not done here.

To get a better estimate on each of these outcome values and their respective errors, we run the simulation 10 times for the same 10,000 number of neutrons to take the mean and standard deviation. This standard deviation is our estimate on the absolute simulation error. Both running multiple trials and simulating a large number of neutrons should in theory make our results more resilient to fluctuations but this is properly explored later when we look at error variation with number of neutrons.

While working with the results we stick with absolute numbers but to print the results, we display them as percentages with errors on those percentages as this makes rates easier to compare between different materials and makes the results independent of neutron number.

In [43]:
# Defining a function run the random walk a certain number of times to return outcomes values with their errors
def random_walk_errors(material, thickness, number_of_neutrons, trials, method='standard', print_output=False):

    # The Woodock method will be discussed later, for now it can be ignored. Just note that if using woodcock, material and
    # thickness must be a list assumed to be of length 2

    # Assigning an array to store all the outcome information from each trial
    outcomes_all_trials = np.empty((0, 3))

    # Running the simulation a certain number of times and adding the results to the array
    for _ in range(trials):
        if method == 'standard':
            total_of_each = random_walk(material, thickness, number_of_neutrons)
        elif method == 'woodcock':
            total_of_each = woodcock_random_walk(material, thickness, number_of_neutrons)
        outcomes_all_trials = np.vstack((outcomes_all_trials, total_of_each))

    # Finding the means and errors of all outcomes
    all_outcome_means = np.mean(outcomes_all_trials, axis=0)
    all_outcome_errs = np.std(outcomes_all_trials, axis=0)

    # Printing the full output if asked for
    if print_output:

        print('--------------------------------')
        if method == 'standard':
            print(f'Thermal neutrons incident on {material.capitalize()}')
            print('---------------------------------')
            print(f'Thickness: {thickness} cm')
        elif method == 'woodcock':
            print(f'Thermal neutrons incident on {material[0].capitalize()} then {material[1].capitalize()}')
            print('---------------------------------')
            print(f'Thicknesses: {thickness[0]} cm and {thickness[1]} cm')

        print(f'Total neutrons: {number_of_neutrons}')
        print(f'Number of trials: {trials}\n')

        # Looping through each outcomes type, finding the percentage and percentage error and printing the result
        for index, outcome_mean in enumerate(all_outcome_means):
            outcome_type = outcome_types[index]
            outcome_err = all_outcome_errs[index]
            outcome_percentage = outcome_mean * 100 / number_of_neutrons
            outcome_percentage_err = outcome_err * 100 / number_of_neutrons

            print(f'Neutrons {outcome_type}: {outcome_mean:.0f} ± {outcome_err:.1f}')
            print(f'Percentage {outcome_type}: {outcome_percentage:.1f} ± {outcome_percentage_err:.2f} %\n')

        # Adding an extra line at the end to better segregate outputs
        print()

    return all_outcome_means, all_outcome_errs

To compare the differences between materials, we can do this for each material and then plot histograms next to each other on the same scale. This gives us a visual sense of what the differences are.

In [44]:
# Defining a function to plot the different outcome values for each material side by side
def plot_material_var(all_means, all_errs):

    # Defining some points to add bar labels to
    bar_coords = np.arange(len(outcome_types))

    # Initialising the figure and a set of subplots based on the number of materials
    fig, axes = plt.subplots(1, len(material_list), sharey=True, figsize=(10, 5))

    # Adding an overarching title
    fig.suptitle('Random walk outcomes for each material')

    # Looping through each material
    for index, material in enumerate(material_list):
        # Defining the current subplot to work in
        ax = axes[index]

        # Storing the relavant mean values for this material
        outcome_means = all_means[index]

        # Plotting the histogram with the data for each outcome for the current material
        ax.bar(bar_coords, outcome_means, align='center', edgecolor='k', yerr=all_errs[index],
               color=outcome_colours)
        # The colorscheme is consistent with that used earlier to plot the random walks.

        # Labeling the outcome types on the x-axis
        ax.set_xticks(bar_coords, outcome_types)

        # Putting the absolute values on top of each bar
        for bar_index, outcome_mean in enumerate(outcome_means):
            ax.text(bar_index, outcome_mean+100, f'{outcome_mean:.0f}', color='k', ha='center')

        # Setting the title
        ax.set_title(f'{material.capitalize()}')

    # Setting the y label only in the first subplot as the axes are shared for all
    axes[0].set_ylabel('Number of neutrons')

    # Displaying the plot
    fig.tight_layout()
    plt.show()

In [45]:
%%time

# Defining a number of neutrons to test
test_number_of_neutrons = 10000

# Creating arrays to store all the means and errors for each material
all_material_means = np.empty((0, len(outcome_types)))
all_material_errs = np.empty((0, len(outcome_types)))

# Looping through each material, finding the mean and error for each and adding it to the array
for test_material in material_list:
    means, errs = random_walk_errors(test_material, initial_thickness, test_number_of_neutrons, number_of_trials,
                                     print_output=True)
    all_material_means = np.vstack((all_material_means, means))
    all_material_errs = np.vstack((all_material_errs, errs))

plot_material_var(all_material_means, all_material_errs)

--------------------------------
Thermal neutrons incident on Water
---------------------------------
Thickness: 10 cm
Total neutrons: 10000
Number of trials: 10

Neutrons reflected: 7956 ± 30.3
Percentage reflected: 79.6 ± 0.30 %

Neutrons absorbed: 2011 ± 31.8
Percentage absorbed: 20.1 ± 0.32 %

Neutrons transmitted: 34 ± 3.2
Percentage transmitted: 0.3 ± 0.03 %


--------------------------------
Thermal neutrons incident on Lead
---------------------------------
Thickness: 10 cm
Total neutrons: 10000
Number of trials: 10

Neutrons reflected: 6199 ± 44.3
Percentage reflected: 62.0 ± 0.44 %

Neutrons absorbed: 980 ± 38.9
Percentage absorbed: 9.8 ± 0.39 %

Neutrons transmitted: 2821 ± 60.1
Percentage transmitted: 28.2 ± 0.60 %


--------------------------------
Thermal neutrons incident on Graphite
---------------------------------
Thickness: 10 cm
Total neutrons: 10000
Number of trials: 10

Neutrons reflected: 6841 ± 52.4
Percentage reflected: 68.4 ± 0.52 %

Neutrons absorbed: 83 ± 10

<IPython.core.display.Javascript object>

CPU times: total: 1.75 s
Wall time: 1.8 s


C:\Users\divij\AppData\Local\Temp\ipykernel_264\907680572.py:27: MatplotlibDeprecationWarning: Passing the minor parameter of set_ticks() positionally is deprecated since Matplotlib 3.2; the parameter will become keyword-only two minor releases later.
  ax.set_xticks(bar_coords, outcome_types)


We can see from this that the transmission rates for water and the absorption rates for graphite are almost zero. Water has the highest reflectance and the highest absorption while graphite marginally has the highest transmission although the transmission rates for graphite and lead are comparable. The error on all of these values is relatively small due to us simulating a lot of neutrons.

Overall this shows that for the purpose of building a shielding layer, which is the main question of this report, water is by far the best material as it offers very little transmission. Additionally, its high reflection rate means it also 'recycles' the neutrons back into the reaction mixture the best.

It should be noted that these numbers, while rounded since they are averages, should still add up to the initial number of neutrons. This is not shown explicitly but within the initial random walk generation, there is a check to make sure this is the case for any one given trial. If there is a problem a warning will be printed and since there is no waring we can assume everything works as expected and any discrepancies here are from rounding errors.

## Investigating error variation

As now mentioned many times now, it is also instructive to look at how the error varies with the number of neutrons simulated. For this we will keep number of trials the same since this was already explored in the very first parts of this project. Additionally, this need only be done for one material and we will use water for this as we have determined it to be the best material for our purposes.

The error used here will be divided by the number of neutrons as we want to see what the error on the outcome of each neutron is. If we used the absolute error instead it would obviously rise with number incident neutrons and wouldn't allow us to see error variation very well. We will also plot our very initial estimate for the error on each outcome which can be given as $\sqrt{N}$ with $N$ as the mean outcome tally overall the trials. This error is estimated of this form using a gaussian distribution since not knowing a priori the probability of each outcome means we cannot use the binomial distribution and besides since we are trialing a large number of neutrons, this is a very good estimate.

In [46]:
def plot_error_var(number_of_neutrons_list, all_simulated_errs, all_estimated_errs):

    fig = plt.figure()
    ax = fig.add_subplot(111)

    for index, outcome in enumerate(outcome_types):
        simulated_errs = all_simulated_errs[:, index] / number_of_neutrons_list
        estimated_errs = all_estimated_errs[:, index] / number_of_neutrons_list

        ax.scatter(number_of_neutrons_list, simulated_errs, label=outcome)
        ax.plot(number_of_neutrons_list, estimated_errs, linestyle='--')

    ax.set_xlabel('number of neutrons')
    ax.set_ylabel('count errors as decimals')

    plt.legend()
    plt.show()

In [47]:
%%time

test_material = 'water'
neutron_var_numbers = np.linspace(1000, 5000, num=10, dtype=int)

# neutron_var_means_10cm = all_materials_means_10cm[0]
# neutron_var_errs_10cm = all_materials_errs_10cm[0]

neutron_var_means = np.empty((0, len(outcome_types)))
neutron_var_errs = np.empty((0, len(outcome_types)))

for test_number_of_neutrons in neutron_var_numbers:
    means, errs = random_walk_errors(test_material, initial_thickness, test_number_of_neutrons, number_of_trials,
                                     print_output=False)
    neutron_var_means = np.vstack((neutron_var_means, means))
    neutron_var_errs = np.vstack((neutron_var_errs, errs))

neutron_var_errs_10cm_predicted = np.sqrt(neutron_var_means)

plot_error_var(neutron_var_numbers, neutron_var_errs, neutron_var_errs_10cm_predicted)

<IPython.core.display.Javascript object>

CPU times: total: 7.47 s
Wall time: 7.83 s


We can see from this that, while a bit messy, the error on the number of outcome count does indeed fall of as about $\sqrt{N}$ although the predicted errors for the number of neutrons reflected doesn't the predicted errors. This might be due to the fact that the count for neutrons reflected in water is by far the highest and so the expected error would be higher but the observed standard deviation might be much lower than this.

Nevertheless, this shows that our errors seem to be decently well estimated and gives a more merit to using more neutrons to sample but does indicate that beyond a certain number, the error doesn't decrease all that much and so in favour of computational time it is not necessary to simulate those extra neutrons. From the graph above, our 10000 neutrons seems to be a goof middle ground and so we continue using it.

## Investigating thickness variation

The final thing we look at how these counts vary with the thickness of the material. The reason we look at this is to allow us to extract an attenuation length for each material which is something not possible analytically. We do this using the same procedure as we did in the 1D case for just absorption before with the same kind of equations but obviously with different values.

In [48]:
# Defining a function to plot thickness variation for every material
def plot_random_walk_thickness_var(material, thicknesses, all_means, all_errs):

    # Initialising the figure and 2 subplots
    fig = plt.figure(figsize=(10, 5))
    ax_1 = fig.add_subplot(121)
    ax_2 = fig.add_subplot(122)

    # Adding an overarching title
    fig.suptitle(f'{material.capitalize()}')

    # Plotting the points for how each outcome count varies on the first subplot
    for index, outcome in enumerate(outcome_types):
        ax_1.errorbar(thicknesses, all_means[:, index], all_errs[:, index] ,label=outcome, marker='.')

    # Isolating the transmission counts and errors
    transmission_means = all_means[:, 2]
    transmission_errs = all_errs[:, 2]

    # Finding the zero counts and removing those points to not get ln(0) errors
    zeros_indices = np.where(transmission_means == 0)[0]
    transmission_means = np.delete(transmission_means, zeros_indices)
    transmission_errs = np.delete(transmission_errs, zeros_indices)
    thicknesses = np.delete(thicknesses, zeros_indices)

    # Finding the ln of the counts and the error on this
    ln_tr_means = np.log(transmission_means)
    ln_tr_means_err = transmission_errs / transmission_means

    # Plotting these points on the second subplot
    ax_2.errorbar(thicknesses, ln_tr_means, yerr=ln_tr_means_err, fmt='x')

    # Calling the function defined near the start to find the best fit lambda, the error on it and the points which give the best fit line
    best_fit_points, lambda_found, lambda_found_err = fit_lambda(thicknesses, ln_tr_means, ln_tr_means_err)

    # Plotting the best fit line on the second subplot
    ax_2.plot(thicknesses, best_fit_points, color='r', label='best-fit line')

    # Labelling the fist subplot
    ax_1.set_xlabel('thickness of material (cm)')
    ax_1.set_ylabel('count')
    ax_1.set_title('Count variation with thickness')

    # Labelling the second subplot
    ax_2.set_xlabel('thickness of material (cm)')
    ax_2.set_ylabel('ln(count)')
    ax_2.set_title('Finding attenuation length using transmittance')

    # Displaying the legend and the plot
    ax_1.legend()
    ax_2.legend()
    plt.tight_layout()
    plt.show()

    # Printing all the results
    print(f'The characteristic attenuation length of {material} is found to be {lambda_found:.2f} ± {lambda_found_err:.2f} cm')


In [49]:
%%time

# Creating a list of thicknesses to test
thickness_var = np.linspace(1, 20, num=20, dtype=int)

# Redefining the number of neutrons to test back to the original number
test_number_of_neutrons = 10000

# Looping through each material
for test_material in material_list:

    # Defining arrays to store the values for each thickness
    thickness_var_means = np.empty((0, len(outcome_types)))
    thickness_var_errs = np.empty((0, len(outcome_types)))

    # Looping through each thickness
    for current_thickness in thickness_var:
        # Finding the mean counts and errors and adding them to the array
        means, errs = random_walk_errors(test_material, current_thickness, test_number_of_neutrons, number_of_trials,
                                         print_output=False)
        thickness_var_means = np.vstack((thickness_var_means, means))
        thickness_var_errs = np.vstack((thickness_var_errs, errs))

    # Calling the function to plot all this data for this material and print the mean free path found
    plot_random_walk_thickness_var(test_material, thickness_var, thickness_var_means, thickness_var_errs)


<IPython.core.display.Javascript object>

The characteristic attenuation length of water is found to be 1.90 ± 0.03 cm


<IPython.core.display.Javascript object>

The characteristic attenuation length of lead is found to be 9.35 ± 0.24 cm


<IPython.core.display.Javascript object>

The characteristic attenuation length of graphite is found to be 10.80 ± 0.53 cm
CPU times: total: 27.8 s
Wall time: 27.8 s


We can see a lot of information from this. Firstly, as expected, as you increase the thickness, the number of neutrons transmitted decreases, the number absorbed increases and the number reflected decreases. Also number reflected and trasnitted seems to be exponential is form which lines up with our theory. Secondly we can see that the absolute error increases as thickness increases which is again something we know intuitively since if the neutrons spend more time in the material, there is more time for random fluctuations to occur giving these higher errors.

Looking at the materials specifically now, we can see that the transmission rate for water hits almost zero the fastest while the rate for lead and graphite is still around 10% at 20cm (lower for graphite). Apart from this, we can see that the curves for graphite and lead are very similar in shape barring the absorption curve as the absorption rate is much higher for lead. This matches what we saw earlier.

Finally, it is important to acknowledge that in all cases, especially for lead and graphite, the log graph seems to show a best fit curve instead of the line we would expect from our theory. This suggests that our results are not very accurate since the transmission rate appears to not be a true exponential which is the only thing that would lead to this kind of behaviour. This means that our accuracy in the mean free path for water is likely the highest and that of graphite is likely the lowest since graphite has the worst shape of all of them while water has the best. The only issue for water is that transmission values beyond 10cm are so small that the error on them is so large but this should have been accounted for in the best fit procedure.

Overall we can say that water has the lowest attenuation length out of the 3 materials and graphite has the highest while also having the highest error which is both evident in both the simulated error and visually from the graphs. We can also definitively conclude that 10cm water is a very good choice to be used as a shielding layer since beyond this, there is very little change in transmission rate so there is arguably no point in wasting more material for such little gain.

In [50]:
# Woodcock ; zero division error and return lambda as infinity

# different geometries
# sanity check

## Simulating multiple materials - Woodcock method

The next logical step is to extend our discussion to 2 materials to see how that affects transmission rate. Instead of doing this using the same method we have been using so far, we use another method called the Woodcock method as this offers a much simpler procedure. The reason we didn't use this before is because the procedure uses EGS sampling which while easier to code is more computationally demanding and increases computational time. This added complexity for better performance was worth the effort for one material but is not worth it for two. It should be noted that both procedures display the same physics and so it doesn't matter what method we use in that sense.

This method is explained in detail within the code below but to outline the main premise, it takes the minimum mean free path between the 2 materials to move the neutrons in both materials using the same methods discussed earlier (exponential distribution). As a result of this the neutrons will never move 'too much' but can move 'too little' and we have to account for this. We do this by introducing the idea of a fictitious step in the material where the mean free path is the highest and in the event of a fictitious step, the neutrons will take another step in the same direction without checking for absorption or scattering.

The probability of the fictitious step is given by:
$${P}(f) = \frac{\Sigma_T - \Sigma_i}{\Sigma_T} = 1 - \frac{\Sigma_i}{\Sigma_T}$$
Where $\Sigma_T$ is the maximum cross-section of both the materials and $\Sigma_i$ the cross-section of material that the fictitious steps are occurring in. If a random uniform number is less that this values, we take the step to be fictitious otherwise we take it to be real. The rest of the calculations are the same as they were before.



In [51]:
def woodcock_random_walk(materials, thicknesses, number_of_neutrons, history=False):

    # Defining the initial number of neutrons to compare to later
    initial_number = number_of_neutrons

    # Finding the cross-sections of both materials
    all_macros = []
    for material in materials:
        macro_total = neutron_data[material]['total macro']
        all_macros.append(macro_total)

    # Finding the maximum cross-section and hence the minimum mean free path 
    macro_max = np.max(all_macros)
    lambda_min = 1 / macro_max
    lambda_min = lambda_min * 10**24
    # Finding the total cross-section
    macro_total_sum = np.sum(all_macros)

    # Fining the thickness of the material and the total thickness
    thickness_1 = thicknesses[0]
    total_thickness = np.sum(thicknesses)

    # Finding the fictitious probability in each material
    prob_fictitious_1 = all_macros[0] / macro_total_sum
    prob_fictitious_2 = all_macros[1] / macro_total_sum

    # Finding the probability of absorption in each material
    prob_absorbed_1 = neutron_data[materials[0]]['absorption probability']
    prob_absorbed_2 = neutron_data[materials[1]]['absorption probability']

    # Finding which material has fictitious steps
    if macro_max == all_macros[0]:
        material_index_with_fictitious_steps = 1
    else:
        material_index_with_fictitious_steps = 0

    # print(f"The material with the fictitious steps is {materials[material_index_with_fictitious_steps]}")

    # Initialising the tallies
    total_absorbed = 0
    total_reflected = 0
    total_transmitted = 0

    # Defining the fist step
    neutrons_in_material_1 = np.zeros((3, number_of_neutrons))
    step_direction_1 = np.vstack((np.ones(number_of_neutrons), np.zeros((2, number_of_neutrons))))
    step_size = random_num_exponential_dist(number_of_neutrons, lambda_min)
    step = step_direction_1 * step_size
    neutrons_in_material_1 = neutrons_in_material_1 + step
    
    # An empty array to store neutrons that go into material 2
    neutrons_in_material_2 = np.empty((3, 0))
    
    # It is also important to keep track of every neutron direction in a similar manner
    step_direction_2 = np.empty((3, 0))
    # The way this is coded this is likely overkill but it does the job - the code has far worse inefficiencies than this

    if history:
        neutron_position_history = np.zeros((3, 1))
        neutron_position_history = np.hstack((neutron_position_history, neutrons_in_material_1))

    # While there are neutrons in any material
    while number_of_neutrons > 0:

        # How many neutrons are there in material 1
        number_of_neutrons_material_1 = len(neutrons_in_material_1[0])

        # Running while there are neutrons in material 1
        while number_of_neutrons_material_1 > 0:

            # Check if any neutrons in material 1 have been reflected and add to the tally as needed
            reflected_indices = np.where(neutrons_in_material_1[0] < 0)[0]
            number_reflected = len(reflected_indices)
            total_reflected = total_reflected + number_reflected
            neutrons_in_material_1 = np.delete(neutrons_in_material_1, reflected_indices, axis=1)
            step_direction_1 = np.delete(step_direction_1, reflected_indices, axis=1)

            # Redefine the number of 'alive' neutrons overall
            number_of_neutrons = number_of_neutrons - number_reflected

            # If any neutrons have crossed the boundary (from 1 to 2), move them to neutrons in material 2
            next_material_indices = np.where(neutrons_in_material_1[0] >= thickness_1)[0]
            if len(next_material_indices) != 0:
                neutrons_moved_material = neutrons_in_material_1[:, next_material_indices]
                neutrons_in_material_2 = np.hstack((neutrons_in_material_2, neutrons_moved_material))
                neutrons_in_material_1 = np.delete(neutrons_in_material_1, next_material_indices, axis=1)

                step_dir_moved_material = step_direction_1[:, next_material_indices]
                step_direction_2 = np.hstack((step_direction_2, step_dir_moved_material))
                step_direction_1 = np.delete(step_direction_1, next_material_indices, axis=1)

            # If material 1 is the one with the longer lambda (there are fictitious steps) then run fictitious steps
            if material_index_with_fictitious_steps == 0:

                # Putting all neutrons into a 'to be checked' array and emptying original 'neutrons in material 1' array when
                # the neutrons take a real step, they can be added back into this array
                neutrons_to_be_checked = neutrons_in_material_1
                number_to_be_checked = len(neutrons_to_be_checked[0])
                neutrons_in_material_1 = np.empty((3, 0))

                step_dir_to_be_checked = step_direction_1
                step_direction_1 = np.empty((3, 0))

                # Run while there are still neutrons to be checked for fictitious steps
                while number_to_be_checked > 0:

                    # Check while neutrons take real steps
                    random_nums_v = np.random.uniform(size=number_to_be_checked)
                    not_fictitious_indices = np.where(random_nums_v < prob_fictitious_1)[0]

                    # For the neutrons that take a real step, move them to back into neutron materials in neutrons in material 1
                    if len(not_fictitious_indices) != 0:
                        neutrons_not_fictitious = neutrons_to_be_checked[:, not_fictitious_indices]
                        neutrons_in_material_1 = np.hstack((neutrons_in_material_1, neutrons_not_fictitious))
                        neutrons_to_be_checked = np.delete(neutrons_to_be_checked, not_fictitious_indices, axis=1)

                        step_dir_not_fictitious = step_dir_to_be_checked[:, not_fictitious_indices]
                        step_direction_1 = np.hstack((step_direction_1, step_dir_not_fictitious))
                        step_dir_to_be_checked = np.delete(step_dir_to_be_checked, not_fictitious_indices, axis=1)

                    # For any neutrons to be checked, move them another step in the same direction as they moved last
                    step_size = random_num_exponential_dist(len(neutrons_to_be_checked[0]), lambda_min)
                    step = step_dir_to_be_checked * step_size
                    neutrons_to_be_checked = neutrons_to_be_checked + step

                    if history:
                        neutron_position_history = np.hstack((neutron_position_history, neutrons_to_be_checked))

                    # Check if any neutrons have been reflected and add to the tally as needed
                    reflected_indices = np.where(neutrons_to_be_checked[0] < 0)[0]
                    number_reflected = len(reflected_indices)
                    total_reflected = total_reflected + number_reflected
                    neutrons_to_be_checked = np.delete(neutrons_to_be_checked, reflected_indices, axis=1)
                    step_dir_to_be_checked = np.delete(step_dir_to_be_checked, reflected_indices, axis=1)

                    # Redefine the number of 'alive' neutrons overall
                    number_of_neutrons = number_of_neutrons - number_reflected

                    # If any neutrons have crossed the boundary (from 1 to 2), move them to neutrons in material 2
                    next_material_indices = np.where(neutrons_to_be_checked[0] >= thickness_1)[0]
                    if len(next_material_indices) != 0:
                        neutrons_moved_material = neutrons_to_be_checked[:, next_material_indices]
                        neutrons_in_material_2 = np.hstack((neutrons_in_material_2, neutrons_moved_material))
                        neutrons_to_be_checked = np.delete(neutrons_to_be_checked, next_material_indices, axis=1)

                        step_dir_moved_material = step_dir_to_be_checked[:, next_material_indices]
                        step_direction_2 = np.hstack((step_direction_2, step_dir_moved_material))
                        step_dir_to_be_checked = np.delete(step_dir_to_be_checked, next_material_indices, axis=1)

                    # Redefine the number to be checked
                    number_to_be_checked = len(neutrons_to_be_checked[0])

                # At the end of these fictitious steps, only neutrons left in material one which took a real step are left

            # Check if these real step neutrons have been absorbed and add to the tally as needed
            random_nums_u = np.random.uniform(size=len(neutrons_in_material_1[0]))
            absorbed_indices = np.where(random_nums_u <= prob_absorbed_1)[0]
            number_absorbed = len(absorbed_indices)
            total_absorbed = total_absorbed + number_absorbed
            neutrons_in_material_1 = np.delete(neutrons_in_material_1, absorbed_indices, axis=1)
            step_direction_1 = np.delete(step_direction_1, absorbed_indices, axis=1)

            # Redefine the number of 'alive' neutrons overall
            number_of_neutrons = number_of_neutrons - number_absorbed

            # Find how many neutrons are left in material 1
            number_of_neutrons_material_1 = len(neutrons_in_material_1[0])

            # Move the remaining neutrons another step
            step_direction_1 = isotropic_vectors(number_of_neutrons_material_1)
            step_size = random_num_exponential_dist(number_of_neutrons_material_1, lambda_min)
            step = step_direction_1 * step_size
            neutrons_in_material_1 = neutrons_in_material_1 + step

            if history:
                neutron_position_history = np.hstack((neutron_position_history, neutrons_in_material_1))

        # Now moving onto material 2, neutrons in material 1 array should be empty at this point

        # How many neutrons are there in material 2
        number_of_neutrons_material_2 = len(neutrons_in_material_2[0])

        # Running while there are neutrons in material 2
        while number_of_neutrons_material_2 > 0:

            # If any neutrons have crossed the boundary (from 2 to 1), move them back to neutrons in material 1
            prev_material_indices = np.where(neutrons_in_material_2[0] < thickness_1)[0]
            if len(prev_material_indices) != 0:
                neutrons_moved_material = neutrons_in_material_2[:, prev_material_indices]
                neutrons_in_material_1 = np.hstack((neutrons_in_material_1, neutrons_moved_material))
                neutrons_in_material_2 = np.delete(neutrons_in_material_2, prev_material_indices, axis=1)

                step_dir_moved_material = step_direction_2[:, prev_material_indices]
                step_direction_1 = np.hstack((step_direction_1, step_dir_moved_material))
                step_direction_2 = np.delete(step_direction_2, prev_material_indices, axis=1)

            # Check is any neutrons have been transmitted right from the start and add to the tally as needed
            transmitted_indices = np.where(neutrons_in_material_2[0] > total_thickness)[0]
            number_transmitted = len(transmitted_indices)
            total_transmitted = total_transmitted + number_transmitted
            neutrons_in_material_2 = np.delete(neutrons_in_material_2, transmitted_indices, axis=1)
            step_direction_2 = np.delete(step_direction_2, transmitted_indices, axis=1)
            # These neutrons will have had to go through both materials in one step which while unlikely is possible

            # Redefine the number of 'alive' neutrons overall
            number_of_neutrons = number_of_neutrons - number_transmitted

            # This order is the same as it was in material one 'reflection' then transmission

            # If material 2 is the one with the longer lambda (there are fictitious steps) then run fictitious steps
            if material_index_with_fictitious_steps == 1:

                # Putting all neutrons into a 'to be checked' array and emptying original 'neutrons in material 2' array when
                # the neutrons take a real step, they can be added back into this array
                neutrons_to_be_checked = neutrons_in_material_2
                number_to_be_checked = len(neutrons_to_be_checked[0])
                neutrons_in_material_2 = np.empty((3, 0))

                step_dir_to_be_checked = step_direction_2
                step_direction_2 = np.empty((3, 0))

                # Run while there are still neutrons to be checked for fictitious steps
                while number_to_be_checked > 0:

                    # Check while neutrons take real steps
                    random_nums_v = np.random.uniform(size=number_to_be_checked)
                    not_fictitious_indices = np.where(random_nums_v < prob_fictitious_2)[0]

                    # For the neutrons that take a real step, move them to back into neutron materials in neutrons in material 2
                    if len(not_fictitious_indices) != 0:
                        neutrons_not_fictitious = neutrons_to_be_checked[:, not_fictitious_indices]
                        neutrons_in_material_2 = np.hstack((neutrons_in_material_2, neutrons_not_fictitious))
                        neutrons_to_be_checked = np.delete(neutrons_to_be_checked, not_fictitious_indices, axis=1)

                        step_dir_not_fictitious = step_dir_to_be_checked[:, not_fictitious_indices]
                        step_direction_2 = np.hstack((step_direction_2, step_dir_not_fictitious))
                        step_dir_to_be_checked = np.delete(step_dir_to_be_checked, not_fictitious_indices, axis=1)

                    # For any neutrons to be checked, move them another step in the same direction as they moved last
                    step_size = random_num_exponential_dist(len(neutrons_to_be_checked[0]), lambda_min)
                    step = step_dir_to_be_checked * step_size
                    neutrons_to_be_checked = neutrons_to_be_checked + step

                    if history:
                        neutron_position_history = np.hstack((neutron_position_history, neutrons_to_be_checked))

                    # If any neutrons have crossed the boundary (from 2 to 1), move them back to neutrons in material 1
                    prev_material_indices = np.where(neutrons_to_be_checked[0] < thickness_1)[0]
                    if len(prev_material_indices) != 0:
                        neutrons_moved_material = neutrons_to_be_checked[:, prev_material_indices]
                        neutrons_in_material_1 = np.hstack((neutrons_in_material_1, neutrons_moved_material))
                        neutrons_to_be_checked = np.delete(neutrons_to_be_checked, prev_material_indices, axis=1)

                        step_dir_moved_material = step_dir_to_be_checked[:, prev_material_indices]
                        step_direction_1 = np.hstack((step_direction_1, step_dir_moved_material))
                        step_dir_to_be_checked = np.delete(step_dir_to_be_checked, prev_material_indices, axis=1)

                    # Check is any neutrons have been transmitted and add to the tally as needed
                    transmitted_indices = np.where(neutrons_to_be_checked[0] > total_thickness)[0]
                    number_transmitted = len(transmitted_indices)
                    total_transmitted = total_transmitted + number_transmitted
                    neutrons_to_be_checked = np.delete(neutrons_to_be_checked, transmitted_indices, axis=1)
                    step_dir_to_be_checked = np.delete(step_dir_to_be_checked, transmitted_indices, axis=1)

                    # Redefine the number of 'alive' neutrons overall
                    number_of_neutrons = number_of_neutrons - number_transmitted

                    # Redefine the number to be checked
                    number_to_be_checked = len(neutrons_to_be_checked[0])

                # At the end of these fictitious steps, only neutrons left in material two which took a real step are left

            # Check if these real step neutrons have been absorbed and add to the tally as needed
            random_nums_u = np.random.uniform(size=len(neutrons_in_material_2[0]))
            absorbed_indices = np.where(random_nums_u <= prob_absorbed_2)[0]
            number_absorbed = len(absorbed_indices)
            total_absorbed = total_absorbed + number_absorbed
            neutrons_in_material_2 = np.delete(neutrons_in_material_2, absorbed_indices, axis=1)
            step_direction_2 = np.delete(step_direction_2, absorbed_indices, axis=1)

            # Redefine the number of 'alive' neutrons overall
            number_of_neutrons = number_of_neutrons - number_absorbed

            # Find how many neutrons are left in material 2
            number_of_neutrons_material_2 = len(neutrons_in_material_2[0])

            # Move the remaining neutrons another step
            step_direction_2 = isotropic_vectors(number_of_neutrons_material_2)
            step_size = random_num_exponential_dist(number_of_neutrons_material_2, lambda_min)
            step = step_direction_2 * step_size
            neutrons_in_material_2 = neutrons_in_material_2 + step

            if history:
                neutron_position_history = np.hstack((neutron_position_history, neutrons_in_material_2))

    # Defining a list of value to be returned
    return_list = [total_reflected, total_absorbed, total_transmitted]

    # Compares the total of the 3 tallies to the initial number of neutrons, if there is a mismatch we print a warning.
    total_number = np.sum(return_list)
    if total_number != initial_number:
        print(f"The initial number of neutrons was taken to be {initial_number} while the final number has come out as "
              f"{total_number}. The total number of neutrons has not been conserved for this run of {thicknesses[0]}cm "
              f"{materials[0]} and {thicknesses[1]}cm {materials[1]}.\n")

    # Adding neutron history to that list if asked for
    if history:
        return_list.append(neutron_position_history)

    return return_list

# I am well aware that this code if far from efficient or even 'complete'. There is a lot of repetition which could have been 
# avoided, and I think the way i have done it here, it wouldn't be too hard to generalise this to more materials either. I 
# haven't done this as I simply did not have the time but I still feel like I should acknowledge it is possible. (my guess is that n materials likely requires some type of recursion loop?)

The way to check this method is to test it using vacumm (infinite mean free path and 0 cross-section) as one material and one known material as the other. These values should be the same as they were for 1 material.

In [52]:
# Defining some vacuum data
neutron_data['vacuum'] = {
    'absorption micro': 0,
    'scattering micro': 0,
    'density': 0,
    'molar mass': 0,
}

# Finding the parameters for this vacuum data
find_material_parameters('vacuum')
find_total_mean_free_path('vacuum')
find_probabilities('vacuum')

In [53]:
%%time

# Testing the method with vacuum and water - 'sanity check'
test_materials_trial_1 = ['vacuum', 'water']
test_thicknesses_trial_1 = [10, 10]

vacuum_water_means, vacuum_water_errs = random_walk_errors(test_materials_trial_1, test_thicknesses_trial_1,
                                                           test_number_of_neutrons, number_of_trials, method='woodcock',
                                                           print_output=True)

# Testing the method with 2 of the same material as another check
test_materials_trial_2 = ['water', 'water']
test_thicknesses_trial_2 = [10, 10]

water_water_means, water_water_errs = random_walk_errors(test_materials_trial_2, test_thicknesses_trial_2,
                                                           test_number_of_neutrons, number_of_trials, method='woodcock',
                                                           print_output=True)

# Trying the method out for a different non-trivial combination
test_materials = ['graphite', 'water']
test_thicknesses = [10, 10]

graphite_water_means, graphite_water_errs = random_walk_errors(test_materials, test_thicknesses, test_number_of_neutrons,
                                                               number_of_trials, method='woodcock', print_output=True)

--------------------------------
Thermal neutrons incident on Vacuum then Water
---------------------------------
Thicknesses: 10 cm and 10 cm
Total neutrons: 10000
Number of trials: 10

Neutrons reflected: 7955 ± 48.9
Percentage reflected: 79.6 ± 0.49 %

Neutrons absorbed: 2011 ± 46.0
Percentage absorbed: 20.1 ± 0.46 %

Neutrons transmitted: 34 ± 6.1
Percentage transmitted: 0.3 ± 0.06 %


--------------------------------
Thermal neutrons incident on Water then Water
---------------------------------
Thicknesses: 10 cm and 10 cm
Total neutrons: 10000
Number of trials: 10

Neutrons reflected: 7952 ± 44.6
Percentage reflected: 79.5 ± 0.45 %

Neutrons absorbed: 2045 ± 44.4
Percentage absorbed: 20.5 ± 0.44 %

Neutrons transmitted: 3 ± 1.6
Percentage transmitted: 0.0 ± 0.02 %


--------------------------------
Thermal neutrons incident on Graphite then Water
---------------------------------
Thicknesses: 10 cm and 10 cm
Total neutrons: 10000
Number of trials: 10

Neutrons reflected: 8447 ± 

In [54]:
%%time

# Plotting each of the cases above to see what is going on visually
plot_random_walk(test_materials_trial_1, test_thicknesses_trial_1, method='woodcock')
plot_random_walk(test_materials_trial_2, test_thicknesses_trial_2, method='woodcock')
plot_random_walk(test_materials, test_thicknesses, method='woodcock')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

CPU times: total: 14.2 s
Wall time: 14 s
